In [196]:
fxobj_raw = """
        move.l  #$00008000,col1(a6)
        move.l  #$00C00000,dstoffx(a6)
        move.l  #$00C00000,dstoffy(a6)
        clr.l   dstoffz(a6)
        move.l  #$00000000,phase1(a6)
        move.l  #$00000400,phase2(a6)
        move.l  #$00140000,i(a6)
        move.l  #$00000400,j(a6)
        move.l  #$00000400,k(a6)
        move.l  #$00000400,zamp(a6)
        move.l  #$00000003,asym_fla(a6)
        move.l  #$001E7540,sine_bas(a6)
        move.l  #$00200000,dy(a6)
        move.l  #$00000008,rsym_ord(a6)
        move.l  #$00080000,rsym_ste(a6)
        move.l  #$00000000,rsym_ist(a6)
        move.l  #$00C00000,rxcen(a6)
        move.l  #$00C00000,rycen(a6)
        move.l  #$00000800,roscale(a6)
        move.l  #$00000000,roscal2(a6)
        move.l  #$00000000,roscalei(a6)
        move.l  #$00000000,drxcen(a6)
        move.l  #$00000000,drycen(a6)
        move.l  #$00001000,cvx(a6)
        move.l  #$00020000,cvy(a6)
        move.l  #$00080000,radius(a6)
        move.l  #$00008000,_i1(a6)
        move.l  #$00008000,_i2(a6)
        move.l  #$00008000,_i3(a6)
        move.l  #$00008000,_i4(a6)
        move.l  #$00008000,_i5(a6)
        move.l  #$00008000,_i6(a6)
        move.l  #$001E7540,wave_2(a6)
        move.l  #$00000000,phase4(a6)
        move.l  #$00021555,phase5(a6)
        move.l  #$00000000,plot_mod(a6)
        move.l  #$01850000,vfb_xsca(a6)
        move.l  #$01850000,vfb_ysca(a6)
        move.l  #$00000000,vfb_angl(a6)
        move.l  #$00C00000,dvf_crot_x(a6)
        move.l  #$00C00000,dvf_crot_y(a6)
        move.l  #$00780000,dvf_deltai(a6)
        move.l  #$00000000,dvf_ws_x(a6)
        move.l  #$00E00000,dvf_ws_y(a6)
        move.l  #$00C00000,vfb_xpos(a6)
        move.l  #$00C00000,vfb_ypos(a6)
        move.l  #$0008F000,colx(a6)
        move.l  #$0008F000,coly(a6)
        move.l  #$00010000,cvx2(a6)
        move.l  #$00010000,cvy2(a6)
        move.l  #$00000000,pixsize(a6)
        move.l  #$00400000,phase4(a6)
        clr.l   thang(a6)
        move.l  #$00000000,info(a6)
        move.l  #$00000000,gpu(a6)
        move.l  #$00000000,vectorx(a6)
        move.l  #$00000000,vectory(a6)
        clr.l   height(a6)
        clr.l   _mtrig(a6)
"""

In [67]:
vlm_equs = [x for x in open("../src/vlm/vlm.inc").read().split('\n') if ";" not in x][:-1]

vlm_vars = {}
for v in vlm_equs:
    xs = v.split("EQU")
    k = xs[0].strip()
    v = int(xs[1].strip()[1:],16)
    vlm_vars[k] = v
    

## Create the fx_obj base

In [226]:
def split_bytes(b):
    bs = []
    for i in range(2,10,2):
        bs += [b[i:i+2]]
    return bs

fx_as = [x.strip()[8:] for x in fxobj_raw.split('\n')[1:-1]]
fxobj_base = [('',i,['00']*4) for i in range(0,256,4)]

for l in fx_as:
    xs = l.split(',')
    v = split_bytes(xs[0]) if len(xs) > 1 else ['00'] * 4
    o = xs[1].split('(')[0] if len(xs) > 1 else xs[0].split('(')[0]
    offset = vlm_vars[o] if o in vlm_vars else int(o[1:],16)
    offset = int(offset/4)
    fxobj_base[offset] = (o,offset*4,v)

fxobj_base += [('',i,['FF']*4) for i in range(256,512,4)]
fxobj_base += [('',i,['00']*4) for i in range(512,768,4)]

osc_raw = """
oscbank:        dc.l $0400000
                dc.l $0011100, $001E7540
                dc.l $0000000, $0000000
                dc.l $0016300, $001E7540
                dc.l $0000000, $0000000
                dc.l $0003000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000C000, $001E7540
                dc.l $0000000, $0000000
                dc.l $0010000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000E000, $001E7540
                dc.l $0000000, $0000000
                dc.l $000C300, $001E7540
                dc.l $0000000, $0000000
                dc.l $0000F310, $001E7540, $00000000
"""
OFFSET = 768
osc_string = ','.join([x[21:] for x in osc_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('oscbank',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

pixcon_raw = """
pixcon:         dc.l $00800000, $00000000
                dc.l $00001600, $00003000, $00FFFF00, $00000000
                dc.l $00040000
"""
OFFSET = 896
osc_string = ','.join([x[21:] for x in pixcon_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('pixcon',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

piycon_raw = """
piycon:         dc.l $00800000, $00000000
                dc.l $00001600, $00003000, $00FFFF00, $00000000
                dc.l $00040000
"""
OFFSET = 896+28
osc_string = ','.join([x[21:] for x in piycon_raw.split('\n')[1:-1]])
osc_vals = ['#$'+x.strip()[1:].rjust(8,'0') for x in osc_string.split(',')]
fxobj_base += [('piycon',(i*4)+OFFSET,split_bytes(x)) for i,x in enumerate(osc_vals)]

fxobj_base += [
     ('adsra', 952, ['0A', '00', '01', '00', 'C0', '00', '05', '00', '00', '04']),
     ('adsrb', 962, ['0B', '00', '01', '00', 'C0', '00', '05', '00', '00', '04']),
     ('adsrb', 972, ['0C', '00', '02', '00', 'C0', '00', '05', '00', '00', '04']),
     ('name', 982, ['00', '00']),
     ('py', 984, ['00', '00', '00', '00']),
     ('px', 988, ['00', '00', '00', '00']),
     ('delayf', 992, ['00', '00']),
     ('delayp', 994, ['00', '00']),
     ('dline', 996, ['00', '00', '00', '00']),
     ('delayt', 1000, ['00', '00']),
     ('delayn', 1002, ['00', '00']),
]


In [245]:
bank_info = [
    (68,783,   "Bank 1"),
    (786,1692, "Bank 2"),
    (1695,2203,"Bank 3"),
    (2206,2813,"Bank 4"),
    (2816,3149,"Bank 5"),
    (3154,3788,"Bank 6"),
    (3791,4379,"Bank 7"),
    (4382,4972,"Bank 8"),
    (4975,5368,"Bank 9"),
    (5371,5880,"Bank 10"),
]

def patch_effect(bs,start=0):
    offset = 0
    effect = [None] * 6144
    for i in range(start,len(bs),2):
        advance = int(bs[i][1:],16)
        val = bs[i+1][1:]
        if offset > 6144:
            break
        offset += advance
        if offset > 6144:
            break
        effect[offset] = val
        offset += 1 
    return (i,effect)

bank_data = open("banks.s",'r').read().splitlines()
compressed_banks = []
bank_texts = []
for s,e,name in bank_info:
    bank_text = bank_data[s:e]
    bank_bytes = ','.join([l[6:37] for l in bank_text]).split(',')
    compressed_banks += [bank_bytes]
    bank_texts += [bank_text]

banks = []
for bank_bytes in compressed_banks:
    offset = 0
    effects = []
    for i in range(0,9):
        offset,effect = patch_effect(bank_bytes,offset)
        effects += [effect]
    banks += [effects]


In [246]:
field_names = {
    "info": {
        "00000000": "<Empty>",
        "00000001": "Draw a polygon object        ",
        "00000002": "Draw 3D starfield            ",
        "00000003": "Draw a ring of pixels        ",
        "00000004": "Digital Video Feedback area  ",
        "00000005": "Wave Surface Thang           ",
        "00000006": "Draw mono bitmap coloured    ",
        "00000007": "Draw mono bitmap i-shaded    ",
        "00000008": "Digital Video Feedback area  ",
        "00000009": "Digital Video Feedback area  ",
        "0000000A": "Colour plasma area type SRCEN",
        "0000000B": "Big Jaguar hardware sprite   ",
        "0000000C": "Draw particle object         ",
        "0000000D": "Do particle motion           ",
        "0000000E": "Spectrum as intensities      ",
    },
    "gpu": {
        "00000000":"Draw 3D Starfield                alpha.gas",
        "00000002":"Draw 3D Starfield                alpha.gas ",
        "00000009":"Draw 3D Starfield                alpha.gas ",
        "00010000":"Digital Feedback Video Area      beta.gas  ",
        "00020000":"Draw a Polygon Object            gamma.gas ",
        "00030000":"Post                             psi.gas   ",
        "00040000":"Colour Plasma Area               delta.gas ",
        "00040002":"Colour Plasma Area               delta.gas ",
        "00040003":"Colour Plasma Area               delta.gas ",
        "00040004":"Colour Plasma Area               delta.gas ",
        "00040009":"Colour Plasma Area               delta.gas ",
        "00050000":"Draw Particle Object             epsilon.gas",
        "00050002":"Draw Particle Object             epsilon.gas",
        "00050003":"Draw Particle Object             epsilon.gas",
        "00050009":"Draw Particle Object             epsilon.gas",
        "00060000":"Do Particle motion               theta.gas ",
        "00070000":"Mono Particle Object             sigma.gas ",
        "00080000":"Matrix object                    tau.gas   ",
        "00090000":"Spectrum as intensities          shu.gas   ",
        "000A0000":"Preamble for each effect.        dbeast.gas",
    },
}

In [247]:
def get_field_name(nm,bs):
    if nm not in field_names:
        return ""
    return field_names[nm][''.join(bs)]

def create_effect_objects(effect, prev_sub_effects):
    sub_effects = []
    for fn,prev_fx_obj in enumerate(prev_sub_effects):
        p = fn * 1024
        fx_obj = prev_fx_obj.copy()
        for e,(n,i,bs) in enumerate(fx_obj):
            lb = len(bs)
            ns = bs.copy()
            for k,j in enumerate(range(i,i+lb)):
                if effect[p+j]:
                    ns[k] = effect[p+j]
            fx_obj[e] = (n,i,ns)
        sub_effects += [fx_obj]
    return sub_effects

out_file = open("banks_new.s",'w')
sub_effects = [fxobj_base]  * 6
for bn, bank in enumerate(banks):
    out_file.write("\n;*********************************************\n")
    out_file.write(f"; Bank {bn+1} \n")
    out_file.write(";*********************************************\n")
    for en,effect in enumerate(bank):
        sub_effects = create_effect_objects(effect, sub_effects)
        for i,sub_effect in enumerate(sub_effects):
            for nm,offset,bs in sub_effect[:64]:
                out_file.write(f"; Bank {bn+1} ".ljust(10) + 
                      f"- Effect {en+1}: Sub-Effect {i+1} - " +
                      f"Byte {offset}-{(offset+len(bs))}:".ljust(13) +
                      f" {nm}".ljust(12) +
                      f"dc.l ${''.join(bs)}  " +
                      f"{get_field_name(nm,bs)}  \n")
    out_file.write('\n'.join(bank_texts[bn]) + '\n')